This code build embeddings for customer and article from text description embeddings. Customer embedding are averaged across the dimension of the embeddings.


In [ ]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

path = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
 
df = pd.read_csv(path, dtype={'article_id':str}).astype(str)

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

In [ ]:
desc = df['detail_desc'].unique()

In [ ]:
%%time
embeds = model(desc)

In [ ]:
desc_embedding_pairs = {i:embed.numpy() for i, embed in zip(desc, embeds)}

In [ ]:
df = df[['article_id', 'detail_desc']]

In [ ]:
df['embeddings'] = df.detail_desc.map(desc_embedding_pairs)

In [ ]:
df = df[['article_id', 'embeddings']]

In [ ]:
f = open('article_embeddings_from_text.pickle', 'wb')
pickle.dump(df.values, f)

In [ ]:
trans = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
                    usecols=['customer_id', 'article_id'],
                   dtype=str)

In [ ]:
trans = trans.merge(df, on='article_id').drop('article_id', axis=1)

In [ ]:
%%time
customer_embeddings = trans.groupby('customer_id').mean()

In [ ]:
f = open('customer_embeddings_from_text.pickle', 'wb')

In [ ]:
pickle.dump(customer_embeddings, f)